## Final Project Submission

Please fill out:
* Student name: Antony Njoroge
* Student pace: Part time
* Scheduled project review date/time: 29th April 2025 
* Instructor name: Sam G. Mwangi
* Blog post URL:


# Business Problem
Your company is expanding in to new industries to diversify its portfolio. Specifically, they are interested in purchasing and operating airplanes for commercial and private enterprises, but do not know anything about the potential risks of aircraft. You are charged with determining which aircraft are the lowest risk for the company to start this new business endeavor. You must then translate your findings into actionable insights that the head of the new aviation division can use to help decide which aircraft to purchase.

## 
Risk in aviation could relate to safety, maintenance costs, operational efficiency, regulatory compliance, or even market demand for certain types of aircraft.



# Data
Civil aviation accidents and selected incidents within the United States, its territories and possessions, and in international waters. The dataset is about accidents, not operational data.The key is to determine which aircraft models have the lowest accident rates.But without knowing the denominator (total flights), it's hard to get a rate.Alternatively, focus on the severity of accidents rather than frequency. 